In [91]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
from helpers import load_pint_data, safe_parse_quantity
import pandas as pd
import pint
from tqdm import tqdm

### Import 

In [92]:
ROOT = Path().cwd().parent
VERSION = "01"
DATA_TYPE = "Test"
ROOT

PosixPath('/Users/thomassimader/Documents/NIRB')

In [93]:
import sys
sys.path.append(str(ROOT))
from ComsolClasses.comsol_classes import COMSOL_VTU
from ComsolClasses.entropy import calculate_S_therm

In [94]:
import_folder = ROOT / "Snapshots" / VERSION 
import_folder.exists()

True

In [95]:
temperatures = np.load(import_folder      / "Exports" / f"{DATA_TYPE}_temperatures.npy" )
temperatures_diff = np.load(import_folder / "Exports" / f"{DATA_TYPE}_temperatures_diff.npy" )

In [96]:
# temperatures[41, -1, :] = temperatures[41, 10, :]

### Plot

In [97]:
comsol_data  = COMSOL_VTU(import_folder / DATA_TYPE / f"{DATA_TYPE}_000.vtu")

In [98]:
N_SNAPS, N_TIMES, N_POINTS = temperatures_diff.shape

colors = px.colors.sample_colorscale("jet", [n/(N_SNAPS) for n in range(N_SNAPS)])

fig = go.Figure()
for idx_snap, temperature_array in enumerate(temperatures_diff):
# customdata=np.stack(([t_h] * len(comsol_data.times), [host_k] * len(comsol_data.times)) , axis=-1),
    input_args = {'x' : list(comsol_data.times.values()),
                #   'customdata' : customdata,
                #   'hovertemplate' : "Time: %{x:.2e}<br>Temperature: %{y:.2f}<br>T_h: %{customdata[0]:.2f}<br><br>host_k: %{customdata[1]:.2e}<br>",
                    'legendgroup' : f"{idx_snap:03d}",
                    'name' : f"{idx_snap:03d}",
                    'line' : dict(color=colors[idx_snap]),
                    'opacity': 0.4}
    fig.add_trace(go.Scatter(y=np.max(temperature_array, axis=1), **input_args))
    fig.add_trace(go.Scatter(y=np.min(temperature_array, axis=1), **input_args))

fig.update_layout(
    xaxis_tickformat='.2e',  # Format x-axis numbers to 2 decimal places
    yaxis_tickformat='.2f',  # Format y-axis numbers to 2 decimal places
    showlegend=False,        # Hide the legend
    title_text=f'Temperature Difference (T-T0) - {DATA_TYPE} Snapshots - Parameter Space {VERSION}', # title of plot
    xaxis_title="Time [s]",    # Optional: label for x-axis
    yaxis_title="Temperature [K]"     # Optional: label for y-axis
)
fig.write_image(import_folder / "Exports" / f"{DATA_TYPE}_MinMaxTemperatures.png")
fig.show()


In [99]:
sim_times_path = import_folder / "Exports" / f"{DATA_TYPE}_sim_times.npy"
sim_times = np.load(sim_times_path)

In [100]:
fig = go.Figure(data=[go.Histogram(
    x=sim_times / 60,
    histnorm='percent',
    name='control', # name used in legend and hover labels
    xbins=dict( # bins used for histogram
        # start=-4.0,
        # end=3.0,
        size=2,
        ),
    marker_color='#EB89B5',
    opacity=0.75,
     texttemplate="%{x}"
    )])

fig.update_layout(
    title_text=f'Simulation Time - {DATA_TYPE} Snapshots - Parameter Space {VERSION}', # title of plot
    xaxis_title_text='Time [min]', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargroupgap=0.1, # gap between bars of the same location coordinates
)
fig.write_image(import_folder / "Exports" / f"{DATA_TYPE}_SimulationTime.png")
fig.show()


### Entropy


In [101]:
param_folder = import_folder / "Exports"
param_files = sorted([path for path in param_folder.rglob(f"{DATA_TYPE}*.csv")])
param_files

[PosixPath('/Users/thomassimader/Documents/NIRB/Snapshots/01/Exports/Test_000_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/Snapshots/01/Exports/Test_001_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/Snapshots/01/Exports/Test_002_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/Snapshots/01/Exports/Test_003_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/Snapshots/01/Exports/Test_004_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/Snapshots/01/Exports/Test_005_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/Snapshots/01/Exports/Test_006_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/Snapshots/01/Exports/Test_007_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/Snapshots/01/Exports/Test_008_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/Snapshots/01/Exports/Test_009_parameters.csv'),
 PosixPath('/Users/thomassimader/Documen

In [102]:
ureg = pint.UnitRegistry()
# ureg.define('@alias degC = degree_Celsius')
ureg.define('@alias degC = degree_Celsius')
# ureg.define("degree_Celsius = degC = degC")
# ureg.define('degree_Celsius = kelvin - 273.15')
"degree_Celsius" in ureg

True

In [ ]:
df = pd.read_csv(param_files[0], index_col = 0)
df.columns

Index(['Unnamed: 0', 'quantity'], dtype='object')

In [ ]:
df['quantity_pint'] = df[df.columns[-1]].apply(lambda x : safe_parse_quantity(x, ureg))
df

,quantity,quantity_pint
Unnamed: 0,,
aspect_ratio_dip,1.0 dimensionless,1.0 dimensionless
aspect_ratio_strike,1.25 dimensionless,1.25 dimensionless
beta,0.00059 / kelvin,0.00059 / kelvin
dip,1.0471975511965976 radian,1.0471975511965976 radian
faul_k_long,9.483242595536296e-18 meter ** 2,9.483242595536296e-18 meter ** 2
faul_k_trans,1.2334510719803156e-16 meter ** 2,1.2334510719803156e-16 meter ** 2
fault_Cp,800.0 meter ** 2 / kelvin / second ** 2,800.0 meter ** 2 / kelvin / second ** 2
fault_k_long,1.5e-13 meter ** 2,1.5e-13 meter ** 2
fault_k_trans,1.5e-13 meter ** 2,1.5e-13 meter ** 2


In [105]:
lambda_therm = (1 - df.loc['host_phi', "quantity_pint"]).magnitude * df.loc['host_lambda', "quantity_pint"].magnitude + \
                df.loc['host_phi', "quantity_pint"].magnitude * (4.2)
T0 = 0.5 * (df.loc["T_h", "quantity_pint"] + (20 + 273.15) * ureg.kelvin)
delta_T = (df.loc['T_h', "quantity_pint"] - (20 + 273.15) * ureg.kelvin)
L = df.loc["H", "quantity_pint"]
print(f"{lambda_therm=}")
print(f"{T0=}")
print(f"{delta_T=}")
print(f"{L=}")

lambda_therm=2.0022
T0=<Quantity(379.318059, 'kelvin')>
delta_T=<Quantity(172.336118, 'kelvin')>
L=<Quantity(4000.0, 'meter')>


In [106]:
entropy_gen_unit = np.zeros((temperatures.shape[0], temperatures.shape[1]))
entropy_gen_number = np.zeros_like(entropy_gen_unit)
comsol_data.mesh.clear_point_data()

for idx_snap in range(temperatures.shape[0]):
    for idx_time in range(temperatures.shape[1]):
        comsol_data.mesh.point_data["temp_field"] = temperatures[idx_snap, idx_time, :]
        cell_mesh = comsol_data.mesh.point_data_to_cell_data()
        cell_mesh.cell_data.keys()
        temp_grad = cell_mesh.compute_derivative("temp_field").cell_data["gradient"]
        s0 = calculate_S_therm(lambda_therm,
                        T0.magnitude,
                        temp_grad)

        s0_total = np.sum(s0 * comsol_data.mesh.compute_cell_sizes()["Volume"])

        s0_characteristic = (lambda_therm * delta_T.magnitude**2) / (L.magnitude**2 * T0.magnitude**2)
        entropy_number = s0_total / s0_characteristic / comsol_data.mesh.volume 
        #
        entropy_gen_unit[idx_snap, idx_time] = s0_total
        entropy_gen_number[idx_snap, idx_time] = entropy_number
        
         



In [107]:
fig = go.Figure()

for idx_snap, entropies in enumerate(entropy_gen_unit):
    fig.add_trace(go.Scatter(x=list(comsol_data.times.values()),
                             y=entropies,
                                mode='lines',
                                name=f"{idx_snap:03d}",
                                opacity=0.4,
                            line=dict(color=colors[idx_snap])
                ))
    
fig.update_layout(
    xaxis_tickformat='.2e',  # Format x-axis numbers to 2 decimal places
    yaxis_tickformat='.2f',  # Format y-axis numbers to 2 decimal places
    showlegend=False,        # Hide the legend
    title=f"Entropy generation - {DATA_TYPE} Snapshots - Parameter Space {VERSION}",       # Optional: add a title
    xaxis_title="Time [s]",    # Optional: label for x-axis
    yaxis_title="Entropy [W/K]"     # Optional: label for y-axis
)

fig.write_image(import_folder / "Exports" / f"{DATA_TYPE}_EntropyGeneration.png")
fig.show()

In [108]:
fig = go.Figure()

for idx_snap, entropies in enumerate(entropy_gen_number):
    fig.add_trace(go.Scatter(x=list(comsol_data.times.values()),
                             y=entropies,
                                mode='lines',
                                name=f"{idx_snap:03d}",
                                opacity=0.4,
                            line=dict(color=colors[idx_snap])
                ))
    
fig.update_layout(
    xaxis_tickformat='.2e',  # Format x-axis numbers to 2 decimal places
    yaxis_tickformat='.2f',  # Format y-axis numbers to 2 decimal places
    showlegend=False,        # Hide the legend
    title=f"Entropy generation number - {DATA_TYPE} Snapshots - Parameter Space {VERSION}",       # Optional: add a title
    xaxis_title="Time [s]",    # Optional: label for x-axis
    yaxis_title="Entropy [-]"     # Optional: label for y-axis
)

fig.write_image(import_folder / "Exports" / f"{DATA_TYPE}_EntropyGenerationNumber.png")
fig.show()